In [1]:
#Import das bibliotecas
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math

In [2]:
# Criação do dicionário que receberá os registros
dic_produtos = {'produto':[], 'preco':[], 'desconto':[], 'link':[]}

# Define os headers que serão utilizados no programa
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}

In [3]:
#FutFanatics - Parte 1
url = 'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=1161&variacao=tamanho_41&range=&order=2&pg=1'

site = requests.get(url, headers=headers)

encoding = site.encoding if 'charset' in site.headers.get('content-type', '').lower() else None
parser = 'html5lib'
soup = BeautifulSoup(site.content, parser, from_encoding=encoding)

totais = soup.find_all('div', class_="search-counter")

total_texto = totais[0].get_text().strip()

index = total_texto.find(' ')

qtd  = total_texto[:index]

ultima_pagina = math.ceil(int(qtd)/40)

print("Quantidade de produtos (FutFanatics):",qtd)
print("Quantidade de páginas (FutFanatics):",ultima_pagina)

Quantidade de produtos (FutFanatics): 667
Quantidade de páginas (FutFanatics): 17


In [4]:
#FutFanatics - Parte 2
for i in range(1, ultima_pagina+1):
    print('FutFanatics - página', i, 'de', ultima_pagina)
    url_pag = f'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=1161&variacao=tamanho_41&range=&order=2&pg={i}'
    site = requests.get(url_pag, headers=headers)
    soup = BeautifulSoup(site.content, parser, from_encoding=encoding)
    produtos = soup.find_all('div', class_=re.compile('product-item'))

    for produto in produtos:
        prod = produto.find('div', class_=re.compile('product-name')).get_text().strip()
        preco = produto.find('div', class_='price').get_text().strip()
        desconto = 0
        descontos = produto.find_all('span', class_=re.compile('discount'))
        if len(descontos) > 0:
            desconto = descontos[0].get_text().strip()
        link = produto.find_all('a', href=True)[0]['href']
        dic_produtos['produto'].append(prod)
        dic_produtos['preco'].append(preco)
        dic_produtos['desconto'].append(desconto)
        dic_produtos['link'].append(link)

FutFanatics - página 1 de 17
FutFanatics - página 2 de 17
FutFanatics - página 3 de 17
FutFanatics - página 4 de 17
FutFanatics - página 5 de 17
FutFanatics - página 6 de 17
FutFanatics - página 7 de 17
FutFanatics - página 8 de 17
FutFanatics - página 9 de 17
FutFanatics - página 10 de 17
FutFanatics - página 11 de 17
FutFanatics - página 12 de 17
FutFanatics - página 13 de 17
FutFanatics - página 14 de 17
FutFanatics - página 15 de 17
FutFanatics - página 16 de 17
FutFanatics - página 17 de 17


In [5]:
# Cria o DataFrame Pandas com os produtos capturados
df = pd.DataFrame(dic_produtos)
df.head()

,produto,preco,desconto,link
0,Chuteira Mizuno Morelia Neo IV Beta Japan Camp...,"R$1.999,90",20%,https://www.futfanatics.com.br/chuteira-mizuno...
1,Chuteira Adidas X Crazyfast Messi Elite FG Cam...,"R$1.799,90",0,https://www.futfanatics.com.br/chuteira-adidas...
2,Chuteira Mizuno Alfa Japan Campo Vermelha,"R$1.649,90",17%,https://www.futfanatics.com.br/chuteira-mizuno...
3,Chuteira Mizuno Morelia Neo Campo Preta e Preta,"R$1.399,90",6%,https://www.futfanatics.com.br/chuteira-mizuno...
4,Chuteira Adidas X Speedportal Society Rosa,"R$1.299,90",13%,https://www.futfanatics.com.br/chuteira-adidas...


In [6]:
# Faz algumas transformações nas colunas capturadas
tabela = df.copy()
tabela['preco'] = tabela['preco'].apply(lambda x: str(x).replace(".",'').replace("R$",''))
tabela['preco'] = pd.to_numeric(tabela['preco'].apply(lambda x: str(x).replace(",",'.')))
tabela['desconto'] = pd.to_numeric(tabela['desconto'].apply(lambda x: str(x).replace("%",'')))
tabela['marca'] = tabela['produto'].astype(str).str.split().str[1]
tabela = tabela.sort_values(by=['desconto'],ascending=False)
tabela.head(15)

,produto,preco,desconto,link,marca
646,Chuteira Umbro Stride Futsal Preta e Branca,99.9,50,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
206,Chuteira Puma Ultra Match IT BDP Futsal Laranja,329.9,49,https://www.futfanatics.com.br/chuteira-puma-u...,Puma
565,Chuteira Umbro Fifty IV Futsal Preta,119.9,47,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
666,Chuteira Oxn Velox 4 Campo Laranja,79.9,46,https://www.futfanatics.com.br/chuteira-oxn-ve...,Oxn
266,Chuteira Umbro Sala V Bump Pro Futsal Verde,269.9,46,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
190,Chuteira Kelme Copa Futsal Marinho e Branca,329.9,45,https://www.futfanatics.com.br/chuteira-kelme-...,Kelme
307,Chuteira Umbro Sala V Premier Futsal Vermelha,219.9,45,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
665,Chuteira Oxn Velox 4 Neo Campo Azul,89.9,43,https://www.futfanatics.com.br/chuteira-oxn-ve...,Oxn
664,Chuteira Umbro Reflect Campo Prata,89.9,43,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
651,Chuteira Oxn Velox 4 Neo Campo Marinho e Rosa,89.9,43,https://www.futfanatics.com.br/chuteira-oxn-ve...,Oxn


In [7]:
# Exporta o resultado para CSV
tabela.to_csv('precoChuteiras.csv', encoding='utf-8', sep=',', index=False)